## Comparing titles acquired from CrossRef API with original title used as search string
## and also checking manually-recorded DOIs in original spreadsheet with those returned by CrossRef.

### Reasoning:
- where title matches orinigal record, the corresponding DOI is correct
- where DOI matches and title matches, even more sure 
- Where DOI matches original in record (having searched using manually recorded title string, DOI correct
- no match on either -- check further



### only the Pandas python library needed this time

In [17]:
import pandas as pd


### First bring in the existing list of publictaions (using the columns needed)

In [18]:
df_json = pd.read_json('./A1out_CrossRef_BRC_Bodlist_BODandCrossRefDois_Oct19.json', orient='records', lines=True)

df_json.head()

,ID,DOI,complete,csv_post_title,crossref_API_out
0,1125,10.1186/s12881-014-0095-4,"&amp; , Fenwick AL, Goos JAC, Rankin J, Lord ...",Apparently synonymous substitutions in FGFR2 a...,{'title': ['Apparently synonymous substitution...
1,1996,10.1183/13993003.00321-2016,", Pattinson KT, Turner MR. A wider pathologic...",A wider pathological network underlying breath...,{'title': ['A wider pathological network under...
2,506,None,"Adib-Samii P, Rost N, Traylor M, Devan W, ...",17q25 Locus is associated with white matter hy...,{'title': ['17q25 Locus Is Associated With Whi...
3,1430,10.1093/annonc/mdu449,"and I. Tomlinson*, Findlay JM, Middleton MR, ...",A systematic review and meta-analysis of somat...,{'title': ['A systematic review and meta-analy...
4,848,None,"Dichgans M, Malik R, KÃ•_nig IR, Rosand J,...",Shared genetic susceptibility to ischemic stro...,{'title': ['Shared Genetic Susceptibility to I...


In [19]:
newdata =pd.read_json(df_json.crossref_API_out.to_json()).transpose()
newdata

,title,DOI
0,[Apparently synonymous substitutions in FGFR2a...,10.1186/s12881-014-0095-4
1,[A wider pathological network underlying breat...,10.1183/13993003.00321-2016
2,[17q25 Locus Is Associated With White Matter H...,10.1161/strokeaha.113.679936
3,[A systematic review and meta-analysis of soma...,10.1093/annonc/mdu449
4,[Shared Genetic Susceptibility to Ischemic Str...,10.1161/strokeaha.113.002707
...,...,...
2373,[Glycogen metabolism has a key role in the can...,10.1007/s00109-015-1377-9
2374,[Visual short-term memory deficits associated ...,10.1093/brain/awu143
2375,[Glycine receptor antibodies in 2 cases of new...,10.1212/nxi.0000000000000016
2376,[Central nervous system neuronal surface antib...,10.1136/jnnp-2011-301237


In [20]:
df_combo =df_json.join(newdata, rsuffix='_API')

In [21]:
df_combo['title_API'] =  df_combo.title.str[0]
df_combo['title_API'].where(df_combo.title_API.str.len()>=10, inplace=True)

In [22]:
df_combo.head()

,ID,DOI,complete,csv_post_title,crossref_API_out,title,DOI_API,title_API
0,1125,10.1186/s12881-014-0095-4,"&amp; , Fenwick AL, Goos JAC, Rankin J, Lord ...",Apparently synonymous substitutions in FGFR2 a...,{'title': ['Apparently synonymous substitution...,[Apparently synonymous substitutions in FGFR2a...,10.1186/s12881-014-0095-4,Apparently synonymous substitutions in FGFR2af...
1,1996,10.1183/13993003.00321-2016,", Pattinson KT, Turner MR. A wider pathologic...",A wider pathological network underlying breath...,{'title': ['A wider pathological network under...,[A wider pathological network underlying breat...,10.1183/13993003.00321-2016,A wider pathological network underlying breath...
2,506,None,"Adib-Samii P, Rost N, Traylor M, Devan W, ...",17q25 Locus is associated with white matter hy...,{'title': ['17q25 Locus Is Associated With Whi...,[17q25 Locus Is Associated With White Matter H...,10.1161/strokeaha.113.679936,17q25 Locus Is Associated With White Matter Hy...
3,1430,10.1093/annonc/mdu449,"and I. Tomlinson*, Findlay JM, Middleton MR, ...",A systematic review and meta-analysis of somat...,{'title': ['A systematic review and meta-analy...,[A systematic review and meta-analysis of soma...,10.1093/annonc/mdu449,A systematic review and meta-analysis of somat...
4,848,None,"Dichgans M, Malik R, KÃ•_nig IR, Rosand J,...",Shared genetic susceptibility to ischemic stro...,{'title': ['Shared Genetic Susceptibility to I...,[Shared Genetic Susceptibility to Ischemic Str...,10.1161/strokeaha.113.002707,Shared Genetic Susceptibility to Ischemic Stro...


In [23]:
df_matched = df_combo.assign(
    title_match = df_combo.csv_post_title.str.strip().str.lower()==df_combo.title_API.str.strip().str.lower(),
    DOI_match = df_combo.DOI.str.strip().str.lower()==df_combo.DOI_API.str.strip().str.lower() )

In [24]:
df_matched.groupby(['title_match']).count()

,ID,DOI,complete,csv_post_title,crossref_API_out,title,DOI_API,title_API,DOI_match
title_match,,,,,,,,,
False,1198,761,1198,1198,1198,1198,1198,902,1198
True,1180,681,1180,1180,1180,1180,1180,1180,1180


In [25]:
df_for_DOI_match =df_matched[df_matched.title_match==False]

df_for_DOI_match.groupby(['DOI_match']).count()

,ID,DOI,complete,csv_post_title,crossref_API_out,title,DOI_API,title_API,title_match
DOI_match,,,,,,,,,
False,842,405,842,842,842,842,842,546,842
True,356,356,356,356,356,356,356,356,356


In [26]:
df_matched.to_csv('./A2out_CrossRef_BRC_matched_and_unmmatched_Oct19.csv')
df_matched.to_json('./A2out_CrossRef_BRC_matched_and_unmmatched_Oct19.json', orient='records', lines=True)

In [27]:
df_unmatched =df_for_DOI_match[df_for_DOI_match.DOI_match==False]
df_unmatched.shape

(842, 10)

In [28]:
df_unmatched.to_csv('./A2out_CrossRef_BRC_unmatched_Oct19.csv')